In [1]:
import numpy as np
from scipy.integrate import solve_ivp

Gamma = 1.4
R_gas = 287
Troposphere_Lapse_Rate = -6.5
Tropopause_Height = 18000
Stratosphere_Lapse_Rate = 2.5
Stratopause_Height = 48000
Ref_Wind_speed = 250.5
Ref_height = 28000
Alpha = 0.143
With_temperature = False
With_wind = False
SpeedOfSound = 343
phi = 0

class ODESystem:
    def __init__(self, phi):
        self.phi = phi
    
    def __call__(self, t, y):
        x_coord = y[0]
        y_coord = y[1]
        z_coord = y[2]
        theta = y[3]

        dx_dt = self.v_x(z_coord) + self.c_s(z_coord) * np.sin(theta) * np.cos(self.phi)
        dy_dt = self.v_y(z_coord) + self.c_s(z_coord) * np.sin(theta) * np.sin(self.phi)
        dz_dt = self.v_z(z_coord) + self.c_s(z_coord) * np.cos(theta)
        dtheta_dt = np.sin(theta) * (self.partial_cs_z(z_coord) + np.cos(self.phi) * self.partial_vx_z(z_coord) + np.sin(self.phi) * self.partial_vy_z(z_coord))

        return [dx_dt, dy_dt, dz_dt, dtheta_dt]

    def temperature(self, z):
        if With_temperature:
            if z <= Tropopause_Height:
                return 300 + (Troposphere_Lapse_Rate / 1000) * z
            elif z > Tropopause_Height and z <= Stratopause_Height:
                return self.temperature(Tropopause_Height) + (Stratosphere_Lapse_Rate / 1000) * (z - Tropopause_Height)
            else:
                return self.temperature(Stratopause_Height)
        else:
            return 0
    
    def v_x(self, z):
        return 0
    
    def v_y(self, z):
        return 0
    
    def v_z(self, z):
        return 0

    def c_s(self, z):
        if With_temperature:
            return np.sqrt(Gamma * R_gas * self.temperature(z))
        else:
            return SpeedOfSound

    def partial_cs_z(self, z):
        if With_temperature:
            dT_dz = Troposphere_Lapse_Rate / 1000 if z <= Tropopause_Height else Stratosphere_Lapse_Rate / 1000
            return 0.5 * np.sqrt(Gamma * R_gas / self.temperature(z)) * dT_dz
        else:
            return 0
    
    def partial_vx_z(self, z):
        return 0
    
    def partial_vy_z(self, z):
        return 0

# Example usage:
# Initial conditions for [x, y, z, theta]
y0 = [0, 0, 0, 0]
t_span = (0, 10)  # Time span for the integration
ode_system = ODESystem(phi)

# Solve the ODE
sol = solve_ivp(ode_system, t_span, y0, method='RK45')

# Accessing the solution
print(sol.t)  # Time points
print(sol.y)  # Solution values


[0.0000e+00 1.0000e-04 1.1000e-03 1.1100e-02 1.1110e-01 1.1111e+00
 1.0000e+01]
[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00]
 [0.000000e+00 3.430000e-02 3.773000e-01 3.807300e+00 3.810730e+01
  3.811073e+02 3.430000e+03]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00]]
